# Load Data

In [1]:
import pandas as pd
import numpy as np
import requests
import apimoex
from tqdm.auto import tqdm
import time


/Users/rualvb1/self/trading-ml-modeling/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


# ==============================================================================
# ПУНКТ 1.1: ГЕНЕРАЦИЯ ПРИМЕРНЫХ ДАННЫХ
# ==============================================================================
# ВАЖНО: Вы замените эту функцию на свой загрузчик данных с МосБиржи.
# Главное, чтобы на выходе был DataFrame с колонками:
# ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']
# 'Date' должна быть в формате datetime.

def generate_mock_data(num_tickers=105, start_date='2013-01-01', end_date='2023-12-31'):
    """Генерирует случайные, но правдоподобные дневные данные по акциям."""
    print(f"Генерация примерных данных для {num_tickers} тикеров...")
    tickers = [f"TICKER_{i}" for i in range(num_tickers)]
    dates = pd.to_datetime(pd.date_range(start_date, end_date, freq='B')) # 'B' - business days
    
    all_dfs = []
    for ticker in tqdm(tickers, desc="Создание тикеров"):
        # Часть активов торгуется не с начала периода
        start_trading_date = np.random.choice(dates[:len(dates)//2])
        ticker_dates = dates[dates >= start_trading_date]
        
        n_days = len(ticker_dates)
        # Имитация с помощью геометрического броуновского движения
        returns = np.random.normal(loc=0.0005, scale=0.02, size=n_days)
        prices = 100 * np.exp(np.cumsum(returns))
        
        # Создание OHLCV
        df = pd.DataFrame(index=ticker_dates)
        df['Ticker'] = ticker
        df['Close'] = prices
        df['Open'] = df['Close'] * np.random.uniform(0.995, 1.005, size=n_days)
        df['High'] = df[['Open', 'Close']].max(axis=1) * np.random.uniform(1.0, 1.02, size=n_days)
        df['Low'] = df[['Open', 'Close']].min(axis=1) * np.random.uniform(0.98, 1.0, size=n_days)
        df['Volume'] = np.random.randint(1_000_000, 100_000_000, size=n_days)
        
        all_dfs.append(df)
        
    full_df = pd.concat(all_dfs).reset_index().rename(columns={'index': 'Date'})
    print("Генерация данных завершена.")
    return full_df

In [ ]:
def download_moex_data(tickers_info):
    """
    Загружает исторические данные для списка тикеров и объединяет их в один DataFrame.

    :param tickers_info: Список словарей с информацией о тикерах (из apimoex.get_index_tickers).
    :return: pd.DataFrame со всеми данными или None, если загрузка не удалась.
    """
    all_history = []
    
    print("Загрузка исторических данных...")
    with requests.Session() as session:
        # Используем tqdm для наглядного прогресс-бара
        for security in tqdm(tickers_info, desc="Загрузка тикеров"):
            ticker = str(security["SECID"])
            
            # Загружаем историю для одного тикера. start='2013-01-01' для ускорения,
            # так как более ранние данные нам не нужны.
            try:
                history = apimoex.get_board_history(
                    session,
                    ticker, start='2013-01-01',
                    columns=("TRADEDATE", "OPEN", "HIGH", "LOW", "CLOSE", "VOLUME")
                    )
                if history:
                    df = pd.DataFrame(history)
                    df['Ticker'] = ticker  # Добавляем колонку с тикером
                    all_history.append(df)                    

                
                # Небольшая пауза, чтобы не перегружать API запросами
                time.sleep(0.1)

            except Exception as e:
                print(f"Не удалось загрузить данные для {ticker}: {type(e)} {e}")

    if not all_history:
        print("Не удалось загрузить данные ни по одному тикеру.")
        return None
    
    return all_history

    

def prepare_moex_data(all_history):
    # Объединяем все загруженные DataFrame в один большой
    full_df = pd.concat(all_history, ignore_index=True)

    # --- Подготовка данных к нужному формату ---
    # 1. Словарь для переименования колонок
    column_map = {
        'TRADEDATE': 'Date',
        'OPEN': 'Open',
        'HIGH': 'High',
        'LOW': 'Low',
        'CLOSE': 'Close',
        'VOLUME': 'Volume'
    }
    full_df.rename(columns=column_map, inplace=True)

    # 2. Выбираем только нужные колонки в правильном порядке
    required_columns = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']
    full_df = full_df[required_columns]
    
    # 3. Преобразуем дату в правильный формат
    full_df['Date'] = pd.to_datetime(full_df['Date'])
    
    # 4. Сортируем для единообразия
    full_df.sort_values(by=['Ticker', 'Date'], inplace=True)
    
    return full_df.reset_index(drop=True)

In [4]:

# # ==============================================================================
# # ОСНОВНОЙ СКРИПТ
# # ==============================================================================

# out_folder = "../data/"

# # --- ШАГ 0 ---
# raw_data = generate_mock_data()


# # --- ШАГ 0.1: Сохранение данных ---
# output_filename = 'moex_raw_data.csv'
# print(f"\nСохранение данных в файл: {output_filename}")
# # Используем index=False, чтобы не записывать индекс pandas в файл
# raw_data.to_csv(out_folder + output_filename, index=False)

# # АЛЬТЕРНАТИВА: Формат Parquet. Он быстрее и занимает меньше места.
# # raw_data.to_parquet('moex_features_and_labels.parquet', index=False)

# print("Все шаги выполнены успешно!")

In [5]:
# ==============================================================================
# ОСНОВНОЙ СКРИПТ
# ==============================================================================

data_folder = "../data/"
output_filename = 'moex_raw_data.csv'

print("--- Этап 1: Загрузка сырых данных с MOEX ---")

# --- ШАГ 1: Получение списка тикеров индекса IMOEX ---
print("Получение списка тикеров из индекса IMOEX...")
with requests.Session() as session:
    # Получаем только акции ('1') из основного режима торгов ('TQBR')
    all_securities = apimoex.get_board_securities(session, board='TQBR', market='shares')
    imoex_tickers = apimoex.get_index_tickers(session, index="IMOEX")

imoex_ticker_names = [t['ticker'] for t in imoex_tickers]

# Фильтруем, чтобы оставить только акции из индекса
securities_to_download = [s for s in all_securities if s['SECID'] in imoex_ticker_names]

if not securities_to_download:
    print("Не удалось получить список тикеров. Проверьте соединение с интернетом.")
    exit()
    
print(f"Найдено {len(securities_to_download)} тикеров в индексе.")

# --- ШАГ 2: Загрузка исторических данных ---
raw_data = download_moex_data(securities_to_download)
raw_data = prepare_moex_data(raw_data)

# --- ШАГ 3: Сохранение данных ---
if raw_data is not None and not raw_data.empty:
    print("\n--- Итоговый DataFrame ---")
    print(raw_data.info())
    print(raw_data.head())
    
    print(f"\nСохранение данных в файл: {data_folder + output_filename}")
    raw_data.to_csv(data_folder + output_filename, index=False)
    print("Сырые данные успешно загружены и сохранены!")
else:
    print("\nИтоговый DataFrame пуст. Файл не будет сохранен.")

--- Этап 1: Загрузка сырых данных с MOEX ---
Получение списка тикеров из индекса IMOEX...
Найдено 78 тикеров в индексе.
Загрузка исторических данных...


Загрузка тикеров:  41%|████      | 32/78 [02:54<03:36,  4.71s/it]

Не удалось загрузить данные для MSTT: HTTPSConnectionPool(host='iss.moex.com', port=443): Max retries exceeded with url: /iss/history/engines/stock/markets/shares/boards/TQBR/securities/MSTT.json?iss.json=extended&iss.meta=off&from=2013-01-01&iss.only=history%2Chistory.cursor&history.columns=TRADEDATE%2COPEN%2CHIGH%2CLOW%2CCLOSE%2CVOLUME&start=300 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1010)')))


Загрузка тикеров: 100%|██████████| 78/78 [06:57<00:00,  5.35s/it]



--- Итоговый DataFrame ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187827 entries, 0 to 187826
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   Date    187827 non-null  datetime64[ns]
 1   Ticker  187827 non-null  object        
 2   Open    185113 non-null  float64       
 3   High    185113 non-null  float64       
 4   Low     185113 non-null  float64       
 5   Close   185113 non-null  float64       
 6   Volume  187827 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 10.0+ MB
None
        Date Ticker    Open    High     Low   Close    Volume
0 2014-06-09   AFKS  44.364  45.001  43.751  44.448   4380200
1 2014-06-10   AFKS  44.440  45.596  44.117  45.499  11586400
2 2014-06-11   AFKS  45.007  45.749  44.700  45.300   4757700
3 2014-06-16   AFKS  45.913  46.370  44.514  45.999  17932600
4 2014-06-17   AFKS  46.300  46.467  45.700  46.100   554450